In [ ]:
import base64
from io import BytesIO
from PIL import Image

import matplotlib.pyplot as plt
import pandas as pd

from run import History

In [ ]:
history_path = "results/winrate.jsonl"

history = History()
history.load_history(history_path)

def decode_image(base64_str: str) -> Image:
    image_data = base64.b64decode(base64_str)
    return Image.open(BytesIO(image_data))

In [ ]:
df = pd.DataFrame(history.entries)
print(df.shape)
df.head()

In [ ]:
# Plot the value counts of the novelty score column in a bar chart with x axis sorted by key
df['novelty_score'].value_counts().sort_index().plot(kind='bar')

In [ ]:
def compare_novelty_trends(history_paths: list[str]):
    fig, ax = plt.subplots()
    for history_path in history_paths:
        history = History()
        history.load_history(history_path)
        df = pd.DataFrame(history.entries)

        avg_novelty = df.groupby("gen")["novelty_score"].mean()
        std_novelty = df.groupby("gen")["novelty_score"].std()

        ax.errorbar(
            avg_novelty.index,
            avg_novelty.values,
            yerr=std_novelty.values,
            marker="o",
            label=history_path)
    
    ax.axhline(y=6, color="gray", linestyle="--")

    ax.set_xlabel("Generation")
    ax.set_ylabel("Average Novelty Score")
    ax.legend()
    ax.set_title("Novelty Score Trend")

compare_novelty_trends(
    ["results/parallel_history.jsonl",
     "results/pillow.jsonl",
     "results/scoreprompt.jsonl"
    ]
)

In [ ]:
for row in history.entries:
    img = decode_image(row["base64_img"])
    display(img)
    print(f"Generation: {row['gen']}")
    print(f"Novelty Score: {row['novelty_score']}")
    print(f"Rationale: {row['rationale']}")